In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip '/content/drive/My Drive/new hope classification/train_tp'
!unzip '/content/drive/My Drive/new hope classification/train_fp1'
!unzip '/content/drive/My Drive/testing/test_tp0.2'
!unzip '/content/drive/My Drive/testing/test_fp0.2'

In [ ]:
import keras
from keras.models import Model
from keras.layers import Dense
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image

import os
import cv2
from PIL import Image
import numpy as np
np.random.seed(1000)
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [ ]:
import os
import cv2
import numpy as np

tp = 0
fp = 0
    
X_train = []
y_train = []
    
path = r'/content/content/train_tp'
    
for image in os.listdir(path):
    pic = cv2.imread(path+'/'+image)
    if np.shape(pic) != (224,224,3):
        pic = cv2.resize(pic, (224,224))
        X_train.append(pic)
        y_train.append(1)
        tp += 1
            
path = r'/content/content/train_fp'
    
for image in os.listdir(path):
        # print(image)
    pic = cv2.imread(path+'/'+image)
    if np.shape(pic) != (224,224,3):
        pic = cv2.resize(pic, (224,224))
        X_train.append(pic)
        y_train.append(0)
        fp += 1



print('train tp',tp)
print('train fp',fp)

tp = 0
fp = 0
X_test = []
y_test = []
    
path = r'/content/content/test_tp0.2'
    
for image in os.listdir(path):
    pic = cv2.imread(path+'/'+image)
    if np.shape(pic) != (224,224,3):
        pic = cv2.resize(pic, (224,224))
        X_test.append(pic)
        y_test.append(1)
        tp += 1
            
path = r'/content/content/test_fp0.2'
    
for image in os.listdir(path):
    pic = cv2.imread(path+'/'+image)
    if np.shape(pic) != (224,224,3):
        pic = cv2.resize(pic, (224,224))
        X_test.append(pic)
        y_test.append(0)
        fp += 1


print('test tp',tp)
print('test fp',fp)




train tp 3115
train fp 3712
test tp 320
test fp 217


In [ ]:
from keras.utils import to_categorical

X_train, X_val, y_train, y_val = train_test_split(X_train, to_categorical(np.array(y_train)), test_size = 0.10, random_state=1000)

In [ ]:
from keras.utils import to_categorical

X_tests, _, y_tests, _ = train_test_split(X_test, to_categorical(np.array(y_test)), test_size = 0.00000010)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_generator = ImageDataGenerator(rescale = 1/255,
                                     zoom_range = 0.3,
                                     horizontal_flip = True,
                                     rotation_range = 30)


train_generators = train_generator.flow(np.array(X_train),
                                       y_train,
                                       batch_size = 20,
                                       shuffle = False)

val_generator = train_generator.flow(np.array(X_val),
                                       y_val,
                                       batch_size = 20,
                                       shuffle = False)


test_generator = ImageDataGenerator(rescale = 1/255)

test_generator = test_generator.flow(np.array(X_tests),
                                     y_tests,
                                     batch_size = 20,
                                     shuffle = False)

In [ ]:
from keras.applications.vgg16 import VGG16
vggmodel = VGG16(weights='imagenet', include_top=True)

In [ ]:
vggmodel.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [ ]:
for layers in (vggmodel.layers)[:19]:
    print(layers)
    layers.trainable = False

In [ ]:
X= vggmodel.layers[-2].output
predictions = Dense(2, activation="softmax")(X)
model_final = Model(input = vggmodel.input, output = predictions)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

model_final.fit_generator(train_generators,
                          steps_per_epoch = len(X_train)/20,
                          epochs = 50,
                          shuffle = False,
                          validation_data= val_generator,
                          callbacks=[
                              EarlyStopping(patience=5,restore_best_weights=True)]
                         )
model_final.save("vgg16.h5")

Epoch 1/50
317/316 [==============================] - 92s 290ms/step - loss: 0.7683 - accuracy: 0.5291 - val_loss: 0.6600 - val_accuracy: 0.5447
Epoch 2/50
317/316 [==============================] - 80s 253ms/step - loss: 0.7060 - accuracy: 0.5657 - val_loss: 0.5669 - val_accuracy: 0.5730
Epoch 3/50
317/316 [==============================] - 80s 254ms/step - loss: 0.6829 - accuracy: 0.5873 - val_loss: 0.5943 - val_accuracy: 0.6128
Epoch 4/50
317/316 [==============================] - 81s 254ms/step - loss: 0.6665 - accuracy: 0.6032 - val_loss: 0.5374 - val_accuracy: 0.6454
Epoch 5/50
317/316 [==============================] - 81s 254ms/step - loss: 0.6580 - accuracy: 0.6146 - val_loss: 0.5945 - val_accuracy: 0.6624
Epoch 6/50
317/316 [==============================] - 80s 254ms/step - loss: 0.6485 - accuracy: 0.6244 - val_loss: 0.6310 - val_accuracy: 0.6454
Epoch 7/50
317/316 [==============================] - 81s 254ms/step - loss: 0.6428 - accuracy: 0.6320 - val_loss: 0.5346 - val_ac

In [ ]:
from keras.models import load_model

model_final = load_model('/content/vgg16.h5')
# !cp -r '/content/vgg16_076.h5' '/content/drive/My Drive/testing/'

In [ ]:
predictions = model_final.predict(test_generator)

In [ ]:

'''
Calculate F-score for Positive Samples 
'''

count = 0
total = 0
correct = 0

tp=0
fp=0
fn=0
tn=0


for i in range(len(y_tests)):

    if predictions[i][0] < predictions[i][1] and y_tests[i][0] == 1.:
        fp += 1
    elif predictions[i][0] > predictions[i][1] and y_tests[i][0] == 0.:
        tn += 1
    elif predictions[i][0] < predictions[i][1] and y_tests[i][0] == 0.:
        tp += 1

    total += 1

print('tp: ',tp)
print('fp: ',fp)
print('tn: ',tn)

print('tp+tn: ', tp+tn)
Precision = tp/(tp+fp)
Recall = tp/(tp+tn)

f_score = (2 * Precision * Recall) / (Precision + Recall)


print('Precision: ',Precision)
print('Recall: ',Recall)
print('f_score: ',f_score)

tp:  313
fp:  190
tn:  6
tp+tn:  319
Precision:  0.6222664015904572
Recall:  0.9811912225705329
f_score:  0.7615571776155717


In [ ]:
'''
Calculate F-score for Negative Samples
'''

count = 0
total = 0
correct = 0

tp= 0
fp=0
fn=0
tn=0


for i in range(len(y_tests)):

    if predictions[i][0] > predictions[i][1] and y_tests[i][0] == 1.:
        tp += 1
    elif predictions[i][0] < predictions[i][1] and y_tests[i][0] == 1.:
        tn += 1
    elif predictions[i][0] > predictions[i][1] and y_tests[i][0] == 0.:
        fp += 1

    total += 1

print('tp: ',tp)
print('fp: ',fp)
print('tn: ',tn)

print('tp+tn: ', tp+tn)
Precision = tp/(tp+fp)
Recall = tp/(tp+tn)

f_score = (2 * Precision * Recall) / (Precision + Recall)


print('Precision: ',Precision)
print('Recall: ',Recall)
print('f_score: ',f_score)

tp:  27
fp:  6
tn:  190
tp+tn:  217
Precision:  0.8181818181818182
Recall:  0.12442396313364056
f_score:  0.21600000000000003


In [ ]:
# Confusion Matrix
[27 190
 6  313]

    precision    recall  f1-score   support

0       0.81      0.12      0.21       217
1       0.62      0.98      0.76       319  